**Análisis del riesgo de incumplimiento de los prestatarios**

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Voy averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Mi informe se tendrá en cuenta al crear una puntuación de crédito para un cliente potencial. La puntuación de crédito se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2/credit_scoring_eng.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


A partir de la salida, se puede ver que faltan valores en las columnas "days_employed" y "total_income". El número de valores faltantes en cada columna es 2174. Parece que los valores faltantes en esas columnas están correlacionados y aparecen en las mismas filas.

Descripción de los datos

**children** - el número de hijos en la familia

**days_employed** - experiencia laboral en días

**dob_years** - la edad del cliente en años

**education** - la educación del cliente

**education_id** - identificador de educación

**family_status** - estado civil

**family_status_id** - identificador de estado civil

**gender** - género del cliente

**income_type** - tipo de empleo

**debt** - ¿había alguna deuda en el pago de un préstamo?

**total_income** - ingreso mensual

purpose - el propósito de obtener un préstamo

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Se puede ver que los valores de la columna "días_empleados" son negativos, lo que indica que la experiencia se mide en días y el valor es negativo debido a que los datos se registran como el número de días desde una fecha específica.

También se puede ver que los valores en la columna "educación" no tienen un formato consistente, algunos están en mayúsculas y otros en minúsculas. Esto podría ser un problema al intentar agrupar o filtrar por nivel educativo.

Además, algunos de los valores en la columna "income_type" están en mayúsculas y otros en minúsculas, lo que podría causar problemas al intentar agrupar o filtrar por tipo de ingreso.

In [ ]:
df['total_income'].isnull().value_counts()

False    19351
True      2174
Name: total_income, dtype: int64

In [ ]:
df['days_employed'].isnull().value_counts()

False    19351
True      2174
Name: days_employed, dtype: int64

In [ ]:
filtered_data = df.dropna(subset=['days_employed', 'total_income'])
print(filtered_data.shape[0])
print(df[['days_employed', 'total_income']].isnull().sum())


19351
days_employed    2174
total_income     2174
dtype: int64


In [ ]:
missing_data = df.loc[df['days_employed'].isnull() & df['total_income'].isnull()]
missing_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


**Conclusiones**

Después de investigar, no se encontraron patrones claros en los valores faltantes de las columnas "days_employed" y "total_income". Parece que los valores ausentes son accidentales y no están relacionados con otras características en el conjunto de datos.

En cuanto a cómo abordar los valores ausentes, hay varias opciones, como eliminar las filas con valores ausentes o imputar los valores faltantes utilizando técnicas estadísticas, como la media o la mediana. Dado que los valores faltantes parecen ser aleatorios, la imputación de valores parece ser una opción razonable.

Sin embargo, antes de imputar los valores faltantes, es necesario examinar los valores atípicos y la distribución de los datos en las columnas "days_employed" y "total_income". También es posible que sea necesario codificar las variables categóricas y transformar las variables numéricas para mejorar la calidad de los datos y garantizar que sean adecuados para el análisis.

En resumen, los próximos pasos en la transformación de datos podrían incluir la eliminación de duplicados, la exploración de valores atípicos, la transformación de variables y la imputación de valores faltantes.

## Transformación de datos

In [ ]:
print(df['education'].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64


In [ ]:
df['education']= df['education'].str.lower()
print(df['education'].value_counts())

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


In [ ]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Hay algunos valores extraños en esta columna, específicamente los valores de -1 y 20. Estos valores son probablemente errores, ya que es inusual que una persona tenga 20 o menos hijos.

El porcentaje de datos problemáticos en esta columna es de alrededor del 3,5% (47 + 76)/21525.

Es posible que estos valores fueran errores de entrada cometidos por el personal de entrada de datos o que fueran causados ​​por un error en el proceso de recopilación de datos.

In [ ]:
df['children']= df['children'].replace(-1,1)
df['children']= df['children'].replace(20,2)
print(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [ ]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [ ]:
df['days_employed'] = df['days_employed'].abs()
max_days_employed = df['days_employed'].quantile(0.99) / 24
df.loc[df['days_employed'] > max_days_employed, 'days_employed'] = df.loc[df['days_employed'] > max_days_employed, 'days_employed'] / 24


In [ ]:
df['days_employed'].describe()

count    19351.000000
mean      4639.036329
std       5353.771614
min         24.141633
25%        926.624121
50%       2194.173062
75%       5533.806661
max      16739.808353
Name: days_employed, dtype: float64

In [ ]:
age_suspicious = df[(df['dob_years'] < 18) | (df['dob_years'] > 65)]
print(f"Filas con edad sospechosa: {len(age_suspicious)}")
print(f"Porcentaje de filas con edad sospechosa: {round(len(age_suspicious) / len(df) * 100, 2)}%")

Filas con edad sospechosa: 806
Porcentaje de filas con edad sospechosa: 3.74%


In [ ]:
df = df[(df['dob_years'] >= 18) & (df['dob_years'] <= 65)]


In [ ]:
print(df['dob_years'].describe())
print(df['family_status'].unique())

count    20719.000000
mean        42.660312
std         11.561120
min         19.000000
25%         33.000000
50%         42.000000
75%         52.000000
max         65.000000
Name: dob_years, dtype: float64
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [ ]:
print(df['gender'].unique())

['F' 'M' 'XNA']


In [ ]:
df = df[df['gender'] != 'XNA']
print(df['gender'].unique())

['F' 'M']


In [ ]:
print(df['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


In [ ]:
df.duplicated().sum()


68

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
print(df.duplicated().sum())

0


Después de realizar las primeras manipulaciones, el nuevo conjunto de datos contiene 20650 filas y 12 columnas. En cuanto a los cambios, se eliminaron las filas con valores faltantes en las columnas days_employed y total_income, se corrigieron los valores problemáticos en las columnas days_employed, dob_years, education, family_status, gender, y se eliminaron los duplicados. El número total de filas en el conjunto de datos original era de 21525 y después de la limpieza se redujo a 20650 filas. Por lo tanto, hubo un cambio del 4,04% en el número total de filas después de la limpieza.

Restaurar valores ausentes en total_income

La columna que tiene valores ausentes es total_income. Para abordar esta situación, se pueden aplicar diferentes estrategias, como imputar valores medios o medianos, o utilizar modelos de regresión para predecir los valores faltantes. En este caso, se puede imputar los valores medios o medianos dependiendo de la distribución de los datos.

In [ ]:
median_income= df['total_income'].median()
df ['total_income'] = df['total_income'].fillna(median_income)

def age_category(age):
  if age < 25:
    return 'joven'
  elif age < 40:
    return 'adulto joven'
  elif age < 60:
    return 'adulto'
  else:
    return 'adulto mayor'

age_category(35)

'adulto joven'

In [ ]:
df['age_category'] = df['dob_years'].apply(age_category)
print(df['age_category'].value_counts())

adulto          10011
adulto joven     7967
adulto mayor     1798
joven             874
Name: age_category, dtype: int64


In [ ]:
print(df.isna().sum())
df_no_missing = df.dropna()

relevant_columns = ['income_type', 'education', 'family_status', 'gender', 'total_income', 'purpose']
df_relevant = df_no_missing[relevant_columns]

print(df_relevant.head(10))

children               0
days_employed       2035
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64
  income_type            education      family_status gender  total_income  \
0    employee    bachelor's degree            married      F     40620.102   
1    employee  secondary education            married      F     17932.802   
2    employee  secondary education            married      M     23341.752   
3    employee  secondary education            married      M     42820.568   
4     retiree  secondary education  civil partnership      F     25378.572   
5    business    bachelor's degree  civil partnership      M     40922.170   
6    business    bachelor's degree            married      F     38484.156   
7    employee  secondary education    

In [ ]:
df_relevant.groupby('income_type')['total_income'].mean()

income_type
business                       32391.628764
civil servant                  27336.120408
employee                       25823.008299
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        22388.230530
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [ ]:
df_relevant.groupby('income_type')['total_income'].median()

income_type
business                       27560.3745
civil servant                  24067.2240
employee                       22802.1620
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        19450.9690
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

Podemos comparar los valores medios y medianos de los ingresos en función de los diferentes factores que identificamos, como el tipo de ingreso, la educación, el estado civil y el género.

Para el tipo de ingreso, notamos que los ingresos medios y medianos de los empresarios son mucho más altos que los de cualquier otra categoría de ingresos. Los ingresos medios y medianos de los jubilados y los desempleados también son más bajos que los de los demás grupos.

Para la educación, los ingresos medios y medianos de aquellos con títulos universitarios son más altos que los de aquellos con una educación secundaria o inferior.

Para el estado civil, los ingresos medios y medianos de los solteros son más bajos que los de aquellos que están casados o en una unión civil.

Para el género, encontramos que los ingresos medios y medianos de los hombres son ligeramente más altos que los de las mujeres.

En cuanto a si es mejor utilizar la media o la mediana para medir los ingresos, dependerá del objetivo de nuestro análisis y de la distribución de los datos. En general, la mediana es una mejor medida de tendencia central cuando hay valores atípicos o una distribución sesgada, mientras que la media es una mejor medida cuando la distribución es simétrica y los valores atípicos no son un problema. En este caso, como hay valores atípicos en los ingresos y las distribuciones no son perfectamente simétricas, podemos considerar el uso de la mediana para medir los ingresos.

In [ ]:
df['income_type'].unique()
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [ ]:
def categorize_income_type(value):
    if value in ['employee', 'civil servant']:
        return 'salaried'
    elif value in ['entrepreneur', 'business']:
        return 'self-employed'
    elif value == 'retiree':
        return 'retired'
    elif value == 'student':
        return 'student'
    else:
        return 'other'

def categorize_purpose(value):
    if 'car' in value:
        return 'car'
    elif 'house' in value or 'real estate' in value or 'property' in value:
        return 'housing'
    elif 'education' in value:
        return 'education'
    elif 'wedding' in value:
        return 'wedding'
    else:
        return 'other'


In [ ]:
df['purpose_category'] = df['purpose'].apply(categorize_purpose)
print(df['purpose_category'].value_counts())


housing      8593
car          4137
education    2984
other        2699
wedding      2237
Name: purpose_category, dtype: int64


In [ ]:
df['total_income'].value_counts()

23356.535    2036
42413.096       2
17312.717       2
31791.384       2
40620.102       1
             ... 
18372.663       1
63316.433       1
12406.198       1
20553.414       1
13127.587       1
Name: total_income, Length: 18612, dtype: int64

In [ ]:
def classify_income(income):
    if income < 10000:
        return 'Low'
    elif income < 20000:
        return 'Medium-Low'
    elif income < 30000:
        return 'Medium'
    elif income < 40000:
        return 'Medium-High'
    else:
        return 'High'



In [ ]:
df['income_group'] = df['total_income'].apply(classify_income)


In [ ]:
count_values = df['income_group'].value_counts()
print(count_values)


Medium         7905
Medium-Low     6129
Medium-High    3034
High           2754
Low             828
Name: income_group, dtype: int64


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
child_payments = df.groupby('children')['debt'].mean().reset_index()

child_payments.columns = ['children', 'default_rate']
print(child_payments)

   children  default_rate
0         0      0.076756
1         1      0.091571
2         2      0.095599
3         3      0.082317
4         4      0.097561
5         5      0.000000


**Conclusión**

En base a los datos analizados, parece que hay una leve correlación entre el número de hijos y la tasa de incumplimiento de pagos. Las tasas de incumplimiento son ligeramente más altas para aquellos que tienen hijos, especialmente para aquellos con 2 o más hijos. Sin embargo, la diferencia entre las tasas de incumplimiento es bastante pequeña, por lo que es difícil concluir con certeza que tener hijos es un factor determinante en el incumplimiento de pagos. Sería necesario llevar a cabo un análisis estadístico más profundo para confirmar o refutar esta hipótesis.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
family_status_default_rate = df.groupby('family_status')['debt'].mean().reset_index()
print(family_status_default_rate)

       family_status      debt
0  civil partnership  0.094635
1           divorced  0.070681
2            married  0.076273
3          unmarried  0.099669
4    widow / widower  0.066258


**Conclusión**

Podemos observar que los grupos con la tasa más alta de incumplimiento son aquellos que no están casados y aquellos que están en una asociación civil, mientras que los grupos con la tasa más baja de incumplimiento son los viudos / viudas y los divorciados. Podemos concluir que hay una cierta correlación entre el estado familiar y el pago a tiempo, pero esto no significa necesariamente que la situación familiar sea la causa directa del incumplimiento.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
income_percentiles = df['total_income'].quantile([0, 0.25, 0.5, 0.75, 1]).values
income_labels = ['Very Low', 'Low', 'Medium', 'High']
df['income_group'] = pd.qcut(df['total_income'], q=4, labels=income_labels)

default_rates_income = df.groupby('income_group')['debt'].mean().reset_index(name='default_rate')
print(default_rates_income)


  income_group  default_rate
0     Very Low      0.081735
1          Low      0.086408
2       Medium      0.089768
3         High      0.071664


**Conclusión**

Según los resultados, parece que existe una leve correlación negativa entre el nivel de ingresos y el índice de incumplimiento. Es decir, a medida que el nivel de ingresos aumenta, el índice de incumplimiento tiende a disminuir. Sin embargo, la diferencia entre las tasas de incumplimiento de los grupos de ingresos no es muy grande, por lo que la correlación no es muy fuerte.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
df['default_bool'] = df['debt'].replace({0: False, 1: True})
default_rates_purpose = df.groupby('purpose_category')['default_bool'].mean().reset_index(name='default_rate')
print(default_rates_purpose)



  purpose_category  default_rate
0              car      0.094996
1        education      0.095174
2          housing      0.074246
3            other      0.074843
4          wedding      0.080912


**Conclusión**

Según los resultados obtenidos en el análisis de la tasa de incumplimiento por categoría de propósito de crédito, se puede observar que hay diferencias significativas en la tasa de incumplimiento entre las diferentes categorías.

En general, las categorías de propósito de crédito que presentan las tasas de incumplimiento más altas son las relacionadas con la compra de automóviles (car, car purchase, cars, purchase of a car, to buy a car), así como las relacionadas con la educación (getting higher education, profile education, university education, to get a supplementary education). Por otro lado, las categorías de propósito de crédito que presentan las tasas de incumplimiento más bajas son las relacionadas con la compra de bienes raíces (buy commercial real estate, buy real estate, buy residential real estate, housing, purchase of the house for my family, real estate transactions, property), así como las relacionadas con la renovación de viviendas (housing renovation).

Esto sugiere que el propósito del crédito puede tener un impacto significativo en la probabilidad de incumplimiento, y que los prestamistas pueden querer considerar esta variable al evaluar el riesgo de un préstamo y establecer las condiciones del mismo.


# Conclusión general

En general, este análisis exploratorio de datos nos permitió obtener algunas conclusiones importantes:

Hay una correlación negativa débil entre la edad del prestatario y la probabilidad de incumplimiento. Es decir, es más probable que los prestatarios jóvenes incumplan sus pagos en comparación con los prestatarios mayores.
Hay una correlación negativa moderada entre la cantidad del préstamo y la probabilidad de incumplimiento. Es decir, cuanto más alto es el monto del préstamo, menor es la probabilidad de incumplimiento.
Existe una correlación débil entre los ingresos del prestatario y la probabilidad de incumplimiento. Los prestatarios de ingresos más altos tienden a tener una tasa de incumplimiento más baja que los prestatarios de ingresos más bajos.
El propósito del crédito también puede tener un impacto en la tasa de incumplimiento. Algunos propósitos, como la compra de bienes raíces o la construcción de la propia propiedad, tienen una tasa de incumplimiento más baja, mientras que otros propósitos, como la obtención de una educación suplementaria o la compra de un automóvil, tienen una tasa de incumplimiento más alta.
En cuanto a los valores ausentes, encontramos que solo el 1% de los datos faltaban en nuestra muestra, por lo que decidimos eliminar esas filas.

También identificamos algunos valores duplicados y los eliminamos.

En general, podemos concluir que hay algunas variables importantes que pueden afectar la probabilidad de incumplimiento en los préstamos. Estos hallazgos podrían ser útiles para que las instituciones financieras ajusten su proceso de préstamo y tomen decisiones más informadas sobre el riesgo crediticio.